In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

This module was written to set up and manipulate cameras (`'bpy.types.Camera'`), scenes (`'bpy.context.scene'`) and other things in Blender from Jupyter without opening it.

This notebook **does not use** a Python kernel, [uses a Blender kernel](https://github.com/panzi/blender_ipython).

In [ ]:
import bpy
import os
from PIL import Image
import pyexiv2

In [ ]:
def open_blend(**kwargs):
    """Open a Blender file to be used with scripts."""
    s3path = kwargs.pop('s3path',
                        os.path.join(os.path.expanduser('~'),
                                     'super-scanner-software-s3'))
    name = kwargs.pop('name', 'phstudio_ArtemisStatue.blend')
    path = kwargs.pop('path',
                      os.path.join(s3path, 'blend-meshes', name))
    try:
        bpy.ops.wm.open_mainfile(filepath=path)
        print('Opened:', path)
    except IOError:
        print('No such file:', path)

In [ ]:
def set_up_sc(**kwargs):
    """Set up a scene ('bpy.context.scene') and return it."""
    wpx = kwargs.pop('wpx', 320)
    hpx = kwargs.pop('hpx', 240)
    antialias = kwargs.pop('antialias', '8')
    bw = kwargs.pop('bw', True)
    cam = kwargs.pop('cam', 'iph4s')
    quality = kwargs.pop('quality', 100)
    extension = kwargs.pop('extension', 'TIFF')
    transparent = kwargs.pop('transparent', False)
    sc = bpy.context.scene
    if transparent:
        sc.render.image_settings.file_format = 'PNG'
        sc.render.alpha_mode = 'TRANSPARENT'
        sc.render.image_settings.color_mode = 'RGBA'
    else:  # 'RGBA' is not supported by JPEG files.
        sc.render.image_settings.file_format = extension
        sc.world.horizon_color = (1, 1, 1)
        if bw:
            sc.render.image_settings.color_mode = 'BW'
        else:
            sc.render.image_settings.color_mode ='RGB'
    sc.render.use_stamp_lens = True
    sc.render.resolution_percentage = quality
    sc.render.resolution_x = wpx
    sc.render.resolution_y = hpx
    sc.render.antialiasing_samples = antialias
    sc.render.use_overwrite = True
    return sc

In [ ]:
def get_camera():
    """Return the first camera ('bpy.types.Camera') of the Blender
    file.('bpy.types.Camera') """
    for i in bpy.data.objects:
        if i.type == 'CAMERA':
            return i
    return -1

In [ ]:
def get_mesh(name, **kwargs):
    """ Return a plane axes that is parent of the first object
    ('MESH')."""
    # Initial location of the axes. The origin is recomended.
    loc0 = kwargs.pop('loc0', (0, 0, 0))
    bpy.ops.object.empty_add(type='PLAIN_AXES', location=loc0)
    axis = bpy.context.active_object
    axis.name = 'axis'
    for obj in bpy.data.objects:
        if obj.name == name:
            obj.parent = axis
            return axis, obj
    return -1

In [ ]:
def set_up_cam(cam, **kwargs):
    """Set up a camera ('bpy.types.Camera') as a real camera.
    
    Add more camera presets to d_cams ('dict') transcribing the values
    from the files in /usr/share/blender/scripts/presets/camera/ to
    the dictionary."""
    idcam = kwargs.pop('idcam', 'iph4s')
    loc = kwargs.pop('loc', (0, 0, 0))
    rot = kwargs.pop('rot', (0, 0, 0))
    d_cams = {'iph4s': ['iPhone 4S', 4.54, 3.42, 4.28, 'HORIZONTAL'], }
    cam.location = loc
    for i in range(len(rot)):
        cam.rotation_euler[i] = rot[i]
    cam.data.sensor_width = d_cams[idcam][1]
    cam.data.sensor_height = d_cams[idcam][2]
    cam.data.lens = d_cams[idcam][3]
    cam.data.sensor_fit = d_cams[idcam][4]

In [ ]:
def num_str_zeros(num, n_digs, matlab=False):
    """Return a string that contains a sequence n-zeros followed by
    num ('int') as 'str', for example, num_str_zeros(89, 4) returns
    '0089'. 
    
    From fwdimaging.ipynb (Python [Root] kernel)."""
    if matlab:  # Begin the numeration with 1 in the filename.
        num += 1
    str_num = ''
    for i in range(n_digs - len(str(num))):
        str_num += '0'
    str_num += str(num)
    return str_num

In [ ]:
def find_out_dir(**kwargs):
    """Return a string that contains a route of a directory.
    
    With replace ('bool') equal to False, this will create a new
    directory if dirname ('string') exists in <s3out>/<parentdir>/"""
    s3out = kwargs.pop('s3out',
                       os.path.join(os.path.expanduser('~'),
                                    's3-out'))
    dirname = kwargs.pop('dirname', 'blend-phg-set-0001')
    parentdir = kwargs.pop('parentdir', 'scanner')
    path = kwargs.pop('path', os.path.join(s3out, parentdir, dirname))
    replace = kwargs.pop('replace', False)
    if not replace:  # Then, create a new direcctory with a different
                     # name.
        while True:
            if not os.path.exists(path):
                print('Make directory:', path)
                os.makedirs(path)
                break
            else:
                str_num = path.split('-')[-1]
                num_dir = int(str_num)
                num_dir += 1
                str_num_dir = num_str_zeros(num_dir, len(str_num))
                path = '-'.join(path.split('-')[:-1]) + '-' + str_num_dir
    return path

In [ ]:
def copy_exif(dest_path, src_path):
    """Copy the Exif metadata of a source image to another."""
    dest_img = Image.open(dest_path)
    wpx = dest_img.size[0]
    hpx = dest_img.size[1]
    dest_img.close()
    src_img = pyexiv2.ImageMetadata(src_path)
    src_img.read()
    dest_img = pyexiv2.ImageMetadata(dest_path)
    dest_img.read()
    src_img.copy(dest_img, exif=True)
    dest_img["Exif.Photo.PixelXDimension"] = wpx
    dest_img["Exif.Photo.PixelYDimension"] = hpx
    dest_img.write()

In [ ]:
def shoot_cam(cam, sc, n_photo, len_img_set, path, **kwargs):
    """Render a scene ('bpy.context.scene') with a camera
    ('bpy.types.Camera') and save the result in outpath
    route (base_file_path, 'string') as image file."""
    s3path = kwargs.pop('s3path',
                        os.path.join(os.path.expanduser('~'),
                                     'super-scanner-software-s3'))
    prefix = kwargs.pop('prefix', 'view_')
    exif = kwargs.pop('exif',
                      os.path.join(s3path,
                                   'img',
                                   'Photo 25-09-16 11 11 00.jpg'))
    base_file_name = prefix + num_str_zeros(n_photo,
                                            len(str(len_img_set)),
                                            matlab=True)
    base_file_path = os.path.join(path, base_file_name)
    sc.render.filepath = base_file_path
    # The next line will save the rendered scene with .tif
    # extension.
    bpy.ops.render.render(write_still=True)
    # The file format is chosen in set_up_sc().
    ext = sc.render.image_settings.file_format
    ext = ext.lower()
    if ext == 'jpeg':
        ext = 'jpg'
    elif ext == 'tiff':
        ext = 'tif'
    file_path = base_file_path + '.' + ext
    # PNG and TIFF/TIF files does not support Exif metadata.
    if ext == 'png' or ext == 'tif':
        print('Saved view without Exif metadata:', file_path)
    else:
        copy_exif(file_path, exif)
        print('Saved view with Exif metadata:', file_path)